# 0809
# 클러스터링으로 tag 채워보기
- 노래 O 태그 X
- 노래 X 태그 O
- 노래 O 태그 O
- 노래 X 태그 X

#### 그외 아이디어
- 가수 이름과 관련된 태그?

#### 수정사항
- 많이나온 순으로 했을 때 count 같은거 어떻게 계산?
---

우선 **`노래 X 태그 O`** 인 경우에 대해서 시도할 예정

---

## 결과 요약
### **`노래 X 태그 O`**에도 종류가 많았음

- 1. 태그가 1개 있는데 그 태그가 train에 존재
    - 놀랍게도 **0건**^^
    
- 2. **태그가 1개 있는데 그 태그가 train에 존재하지 않음** `error_list`

- 3. 태그가 2~9개 있는데 그중 하나라도 train에 존재
    - 나머지 태그중 가장 많이 나온 집합에서 상위 태그 넣었다.
    
- 4. **태그가 2~9개 있는데 모두다 train에 없음** `error_list3`

- 5. 태그가 10개가 채워져 있음

#### 2,4번은 어케해결하지..?
일단 3번만 했다. `perform`이 수행한 거 목록



---

# 1. Data 불러오기

In [1]:
from sklearn import datasets
import pandas as pd
import json
import re # regular expression
import gc # to collect memory
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import matplotlib
import matplotlib.pyplot  as plt
import seaborn as sns

In [43]:
# load json files
with open("./res/genre_gn_all.json", encoding = 'utf-8') as json_file: genre_gn_all = json.load(json_file)
#with open("./res/song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
#with open("./res/train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
with open("./res/val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)

# json to dataframe
genre_gn_all = pd.Series(genre_gn_all).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
song_meta = pd.read_json('./res/song_meta_cnt.json', orient='table')
train = pd.read_json('./res/train_topgnr.json', orient='table')
valid = pd.DataFrame(valid)
tag_cnt=pd.read_json('./res/tag_cnt.json', orient='table')

#지민이가 준 파일 table로 안불러짐.... frame으로 하면 열리긴 하지만 내용깨짐
#song_artist_tag=pd.read_json('./res/song_artist_tag.json', orient='table')

tag_tf_gnrmap=pd.read_json('./res/tag_tf_gnrmap.json', orient='table')

# 2. k-means Clustering

In [48]:
from sklearn.cluster import KMeans

In [49]:
tag_tf_gnrmap

,tags,mapping_cnt,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,...,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000
0,,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,00,4,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,007,3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,007시리즈,2,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,00s,3,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29155,힙해,5,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29156,힙힙힙,2,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29157,힛뎀포크,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29158,힛뎀폭,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [50]:
feature=tag_tf_gnrmap[['GN0100','GN0200','GN0300','GN0400','GN0500','GN0600','GN0700','GN0800','GN0900','GN1000',
                        'GN1100','GN1200','GN1300','GN1400','GN1500','GN1600','GN1700','GN1800','GN1900','GN2000',
                       'GN2100','GN2200','GN2300','GN2400','GN2500','GN2600','GN2700','GN2800','GN2900','GN3000','GN9000']]
feature

,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,GN0900,GN1000,...,GN2200,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29155,False,False,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
29156,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29157,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
29158,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [79]:
model = KMeans(n_clusters=100,algorithm='auto')
model.fit(feature)
predict = pd.DataFrame(model.predict(feature))
predict.columns=['predict']

In [80]:
r = pd.concat([feature,predict],axis=1)

In [81]:
labels = model.labels_
centers = model.cluster_centers_

In [82]:
tag_tf_gnrmap['labels']=labels

In [83]:
tag_tf_gnrmap
#라벨이 추가된걸 확인 가능하다.

,tags,mapping_cnt,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,...,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000,labels
0,,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,10
1,00,4,False,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,99
2,007,3,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
3,007시리즈,2,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,6
4,00s,3,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29155,힙해,5,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,40
29156,힙힙힙,2,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,8
29157,힛뎀포크,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,10
29158,힛뎀폭,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,10


In [102]:
#정렬
tag_tf_gnrmap=tag_tf_gnrmap.sort_values('mapping_cnt', ascending = False)
tag_tf_gnrmap

,tags,mapping_cnt,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,...,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000,labels
7065,기분전환,16465,True,True,True,True,True,True,True,True,...,False,True,True,True,True,True,True,False,False,26
5234,감성,11417,True,True,True,True,True,True,True,True,...,False,True,True,True,True,True,False,False,True,26
28675,휴식,11215,True,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,False,True,26
12894,발라드,10796,True,True,True,True,True,True,True,True,...,False,True,True,False,False,False,False,False,False,98
22262,잔잔한,10218,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,True,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24860,춥고,1,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,1
21635,인더스트리얼테크노,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,5
24857,춤추며,1,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,13
24856,춤추기좋은팝송,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,5


In [110]:
tag_tf_gnrmap.loc[tag_tf_gnrmap['labels']==11].head(10)

,tags,mapping_cnt,GN0100,GN0200,GN0300,GN0400,GN0500,GN0600,GN0700,GN0800,...,GN2300,GN2400,GN2500,GN2600,GN2700,GN2800,GN2900,GN3000,GN9000,labels
28187,협주곡,39,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
6386,교향곡,36,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
16377,스코어,34,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
15493,성악,32,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
12749,바흐,32,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
25673,클래식명곡,32,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
13351,베토벤,32,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
15976,쇼팽,30,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
11924,명연,28,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11
7015,기도,28,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,11


In [105]:
tag_tf_gnrmap_count=tag_tf_gnrmap.groupby('labels').tags.nunique().reset_index(name='count')
tag_tf_gnrmap_count.head(50)

,labels,count
0,0,145
1,1,1665
2,2,1719
3,3,51
4,4,3257
5,5,1571
6,6,2818
7,7,83
8,8,461
9,9,590


In [106]:
tag_tf_gnrmap_count.tail(50)

,labels,count
50,50,45
51,51,92
52,52,77
53,53,238
54,54,75
55,55,32
56,56,125
57,57,97
58,58,65
59,59,139


### 100개로 나뉘었고, 가장적은게 14개인걸로 봐서 10개 채우는 태그에서 충분히 사용 가능할 것으로 예상된다

---

# 3. 노래 없고 태그만 있는것은 클러스터링으로 채워보자

In [321]:
#재실행 위한 코드
with open("./res/val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)
valid = pd.DataFrame(valid)

In [322]:
valid_tago_songx=valid
valid_tago_songx

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [323]:
#index 찾기 test

#valid_tago_songx['tags'][23010][0] : 잔잔한
#tag_tf_gnrmap[tag_tf_gnrmap['tags']==valid_tago_songx['tags'][23010][0]]# : tags 잔잔한 있는 배열
i=tag_tf_gnrmap[tag_tf_gnrmap['tags']==valid_tago_songx['tags'][23010][0]]['labels'].to_frame()
i=i.reset_index()
i['labels'][0]
can_list=tag_tf_gnrmap.loc[tag_tf_gnrmap['labels']==i['labels'][0]].head(11).reset_index()
testlist=['잔잔한']
for j in range(0,11):
    if(len(testlist)==10):
        break;
    testlist.append(can_list['tags'][j])
    
    
testlist

['잔잔한', '기분전환', '감성', '휴식', '잔잔한', '드라이브', '힐링', '사랑', '새벽', '밤']

In [324]:

#max test
testlist2=['안녕','하세요','안녕','안녕']

testlist3=[1,1,2,2,3,3,3]

testlist4=[1,2,3,4,5]
print(max(testlist2),max(testlist3),max(testlist4))



하세요 3 5


### 문자엔 적용안되고, 중복있을땐 맨 마지막 값이 나오는듯???

### 일단 이걸 쓰고 중복은 보강하는 방향으로 해야할듯..

In [325]:
# 노래 X 태그 O 검색 방법 -- 세어보니 2630개, 그중 태그 1개만 딱 있는건 41개, 2~9개 1539개, 10개 꽉 채워진경우 1050개
# 빈배열이 있다고해서 isnull로 걸러지지 않는다ㅠ
error_list=[] #노래0개 태그 1개일때 오류 리스트
error_list2=[] #노래x태그만 있는 것들 중 train에 없는 태그가 있는 경우
error_list3=[] #노래x 태그가 여러개 있으나 단 하나도 train에 없다!

perform=[]

for i in range(0,23015):
    if valid_tago_songx['tags'][i] and not valid_tago_songx['songs'][i]:#tag가 있고, song이 비어 있을 때
        
        #태그 10개 채워야함, tl=tag_length
        need_tl=10-len(valid_tago_songx['tags'][i])
        
        #태그가 딱 1개 있을 때: 
        if(need_tl==9):
            
            try:
                #### 주의사항 top10에 1개있는 태그가 있을수도있고 없을수도 있음.-->set을 이용하면 배열에서 중복없애기 가능
                num=tag_tf_gnrmap[tag_tf_gnrmap['tags']==valid_tago_songx['tags'][i][0]]['labels'].to_frame()
                num=num.reset_index()
                #num['labels'][0]
                can_list=tag_tf_gnrmap.loc[tag_tf_gnrmap['labels']==num['labels'][0]].head(11).reset_index()
                for j in range(0,11):
                    if can_list['tags'][j] not in valid_tago_songx['tags'][i]: #포함되지 않으면
                        valid_tago_songx['tags'][i].append(can_list['tags'][j])
                    
                    # 순서가 섞이지 않게 하되 중복검사를 해야하므로 set적용한걸로 개수를 세어본다.(원본 안바뀜)
                    if(len(list(set(valid_tago_songx['tags'][i])))==10): 
                        break;
                    
            except:
                error_list.append(i)
        
        ##태그가 있긴 하나 8~1개만 더 채워넣으면 되는 경우
        elif(need_tl>0):  
            nlist=[]
            try:
                for p in range(0,len(valid_tago_songx['tags'][i])):
                    try:
                        num=tag_tf_gnrmap[tag_tf_gnrmap['tags']==valid_tago_songx['tags'][i][p]]['labels'].to_frame()
                        num=num.reset_index()
                        nlist.append(num['labels'][0])
                    except:
                        if i not in error_list2: #중복인 경우 제외
                            error_list2.append(i)
                
                can_list=tag_tf_gnrmap.loc[tag_tf_gnrmap['labels']==max(nlist)].head(11).reset_index()
                for j in range(0,11):
                    if can_list['tags'][j] not in valid_tago_songx['tags'][i]: #포함되지 않으면
                        valid_tago_songx['tags'][i].append(can_list['tags'][j])
                        
                    # 순서가 섞이지 않게 하되 중복검사를 해야하므로 set적용한걸로 개수를 세어본다.(원본 안바뀜)
                    if(len(list(set(valid_tago_songx['tags'][i])))==10): 
                        break;
                perform.append(i)    
                    
            except: #혹시모를것 대비(비어있어야정상!)
                error_list3.append(i)
            
            #print(nlist)
            #출력해본 결과 집합이 중복되는 것이 꽤 많았다. 따라서 배열에서 가장 많은 집합을 기준으로 삼고자 한다.
valid_tago_songx

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


---
# 결과 확인해보기

In [326]:
#작동한경우
perform

[20,
 40,
 43,
 49,
 55,
 56,
 58,
 81,
 82,
 96,
 108,
 136,
 139,
 140,
 181,
 209,
 215,
 226,
 266,
 267,
 277,
 283,
 302,
 337,
 340,
 356,
 365,
 366,
 380,
 387,
 397,
 404,
 405,
 409,
 411,
 422,
 458,
 461,
 477,
 484,
 499,
 527,
 528,
 531,
 552,
 557,
 560,
 599,
 604,
 619,
 623,
 625,
 658,
 661,
 700,
 702,
 711,
 721,
 730,
 773,
 782,
 801,
 806,
 811,
 883,
 884,
 896,
 897,
 916,
 927,
 948,
 979,
 987,
 988,
 989,
 996,
 1010,
 1020,
 1061,
 1096,
 1101,
 1106,
 1109,
 1124,
 1143,
 1167,
 1168,
 1174,
 1177,
 1196,
 1205,
 1209,
 1244,
 1249,
 1257,
 1264,
 1298,
 1303,
 1304,
 1313,
 1323,
 1342,
 1367,
 1368,
 1400,
 1408,
 1411,
 1412,
 1432,
 1441,
 1463,
 1468,
 1470,
 1485,
 1525,
 1563,
 1566,
 1575,
 1602,
 1616,
 1624,
 1679,
 1683,
 1692,
 1698,
 1724,
 1725,
 1778,
 1780,
 1787,
 1814,
 1815,
 1816,
 1834,
 1837,
 1856,
 1871,
 1881,
 1888,
 1890,
 1897,
 1907,
 1912,
 1948,
 1955,
 1958,
 1970,
 1975,
 1976,
 2001,
 2012,
 2015,
 2019,
 2028,
 2031,
 

In [332]:
valid_tago_songx['tags'][81]

['달래주는', '연인', '연애', '사랑', '프로포즈', '힙합', '취향저격', '저녁', '집중', '감각적인']

들어간걸 확인 가능

---

In [327]:
#태그가 여러개 있었어나 단 하나도 안맞는경우
error_list3

[160, 573, 1133, 5450, 6928, 13373, 14740, 19162, 21182, 22841]

In [336]:
valid_tago_songx['tags'][6928]

['가슴답답_할_때', '스트레스해소곡', '가슴뿡뚫리는곡']

---

In [328]:
error_list2 #error_list3에서 걸리지 않는다면 괜찮다...

[20,
 160,
 477,
 484,
 528,
 573,
 721,
 884,
 916,
 1133,
 1174,
 1196,
 1368,
 1408,
 1411,
 1525,
 1780,
 1815,
 1888,
 1955,
 1975,
 2019,
 2234,
 2249,
 2438,
 2581,
 2720,
 3012,
 3105,
 3280,
 3605,
 3741,
 3779,
 3788,
 3868,
 3935,
 4188,
 4492,
 4589,
 4667,
 4735,
 4899,
 5248,
 5272,
 5278,
 5318,
 5450,
 5788,
 5910,
 5950,
 5997,
 6454,
 6721,
 6928,
 7022,
 7353,
 7493,
 7912,
 8272,
 8281,
 8283,
 8314,
 8418,
 8537,
 8670,
 8983,
 9016,
 9212,
 9227,
 9379,
 9695,
 9730,
 9750,
 9913,
 9918,
 10076,
 10185,
 10505,
 10606,
 10846,
 10939,
 11096,
 11109,
 11146,
 11161,
 11562,
 11708,
 11747,
 12074,
 12085,
 12900,
 12931,
 13373,
 13545,
 13794,
 13874,
 13908,
 13975,
 14027,
 14308,
 14740,
 14832,
 14848,
 14857,
 14881,
 14937,
 15003,
 15158,
 15472,
 15570,
 15741,
 15792,
 15810,
 16158,
 16487,
 16596,
 16732,
 17079,
 17188,
 17249,
 17295,
 17703,
 17735,
 17744,
 17808,
 18187,
 18294,
 18330,
 18354,
 18397,
 18477,
 18680,
 18913,
 18959,
 18994,
 1916

---

In [337]:
## valid에 tag 1개있는것 중 train에 없던 tag들
error_list

[104,
 109,
 659,
 684,
 1744,
 2346,
 2831,
 3427,
 4048,
 4149,
 4628,
 5693,
 6060,
 6723,
 7713,
 7962,
 9130,
 9413,
 10035,
 10424,
 11069,
 11129,
 11178,
 11346,
 12372,
 12593,
 12723,
 14674,
 15183,
 16408,
 17428,
 18105,
 18249,
 18900,
 20641,
 20881,
 20888,
 21759,
 22133,
 22549,
 22906]

In [273]:
#없는애 예시
valid_tago_songx['tags'][6060]

['청춘2부작']

In [276]:
valid_tago_songx[6060:]

,tags,id,plylst_title,songs,like_cnt,updt_date
6060,[청춘2부작],48551,화양연화 pt.2,[],0,2017-03-23 09:44:06.000
6061,"[Metal, M에센셜, 메탈]",24470,,"[224031, 103782, 567488, 328751, 398547, 45503...",11,2016-07-06 16:27:11.000
6062,"[힐링, 밤]",91765,,"[477964, 120626, 274716, 231563, 71243, 400157...",8,2014-08-06 11:53:28.000
6063,[남자싱어송라이터],5270,,"[405855, 369146, 685550, 27208, 170937, 117402...",1391,2017-04-10 12:33:48.000
6064,[],116661,,"[625200, 501822, 532347]",0,2017-09-25 14:38:56.000
...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000
